In [1]:
%load_ext autoreload
%autoreload 2
import notebook_setup
from src.config import PROCESSED_DATA_DIR
import os

2025-09-22 21:11:55.679 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: D:\workspace\projects\freelance\Fusion3DNet


Project root added to path: d:\workspace\projects\freelance\Fusion3DNet


In [2]:
from pathlib import Path
FEATURES_DIR = PROCESSED_DATA_DIR / "dataset_129" / 'features'
BREPNET_NPZ_DIR = FEATURES_DIR / "brep"

DINOs14_2d_NPZ_DIR = FEATURES_DIR / '2D' / "dinov2_vits14"
DINOb14_2d_NPZ_DIR = FEATURES_DIR / '2D' / "dinov2_vitb14"
CLIPs16_2d_NPZ_DIR = FEATURES_DIR / '2D' / "ViT-B_16"
CLIPb32_2d_NPZ_DIR = FEATURES_DIR / '2D' / "ViT-B_32"

DINOs14_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "dinov2_vits14"
DINOb14_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "dinov2_vitb14"
CLIPs16_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "ViT-B_16"
CLIPb32_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "ViT-B_32"

def get_files(step_path: Path, extensions=("stp", "step")):
    return [f for ext in extensions for f in step_path.glob(f"**/*.{ext}")]

brepnet_files = get_files(BREPNET_NPZ_DIR, ('npz',))
print(f"Количество файлов BrepNet: {len(brepnet_files)}")

dino_s14_2d_files = get_files(DINOs14_2d_NPZ_DIR, ('npz',))
dino_b14_2d_files = get_files(DINOb14_2d_NPZ_DIR, ('npz',))
clip_s16_2d_files = get_files(CLIPs16_2d_NPZ_DIR, ('npz',))
clip_b32_2d_files = get_files(CLIPb32_2d_NPZ_DIR, ('npz',))

dino_s14_2dv0_files = get_files(DINOs14_2dv0_NPZ_DIR, ('npz',))
dino_b14_2dv0_files = get_files(DINOb14_2dv0_NPZ_DIR, ('npz',))
clip_s16_2dv0_files = get_files(CLIPs16_2dv0_NPZ_DIR, ('npz',))
clip_b32_2dv0_files = get_files(CLIPb32_2dv0_NPZ_DIR, ('npz',))

print(f"Количество файлов DINO s14 2d: {len(dino_s14_2d_files)}")
print(f"Количество файлов DINO b14 2d: {len(dino_b14_2d_files)}")
print(f"Количество файлов CLIP s16 2d: {len(clip_s16_2d_files)}")
print(f"Количество файлов CLIP b32 2d: {len(clip_b32_2d_files)}")
print(f"Количество файлов DINO s14 2dv0: {len(dino_s14_2dv0_files)}")
print(f"Количество файлов DINO b14 2dv0: {len(dino_b14_2dv0_files)}")
print(f"Количество файлов CLIP s16 2dv0: {len(clip_s16_2dv0_files)}")
print(f"Количество файлов CLIP b32 2dv0: {len(clip_b32_2dv0_files)}")

Количество файлов BrepNet: 129
Количество файлов DINO s14 2d: 129
Количество файлов DINO b14 2d: 129
Количество файлов CLIP s16 2d: 129
Количество файлов CLIP b32 2d: 129
Количество файлов DINO s14 2dv0: 129
Количество файлов DINO b14 2dv0: 129
Количество файлов CLIP s16 2dv0: 129
Количество файлов CLIP b32 2dv0: 129


In [3]:

from src.modeling.vit.metrics import eval_object_max
import numpy as np

def load_npz_features(npz_path):
    data = np.load(npz_path)
    features = data['views']
    return features


model_names = [
    "DINO s14 2d", "DINO b14 2d", "CLIP s16 2d", "CLIP b32 2d",
    "DINO s14 2dv0", "DINO b14 2dv0", "CLIP s16 2dv0", "CLIP b32 2dv0"
]

model_files = [
    dino_s14_2d_files, dino_b14_2d_files, clip_s16_2d_files, clip_b32_2d_files,
    dino_s14_2dv0_files, dino_b14_2dv0_files, clip_s16_2dv0_files, clip_b32_2dv0_files
]

def evaluate_models_for_search(model_names, model_files, load_fn):
    rows = []
    for name, files in zip(model_names, model_files):
        if not files:
            rows.append({"model": name, "queries": 0})
            continue

        m = eval_object_max(files, load_fn)
        m["model"] = name
        rows.append(m)
    return rows

rows = evaluate_models_for_search(model_names, model_files, load_npz_features)

In [ ]:

from src.modeling.BRepNet.metrics import evaluate_brepnet_faces_object_max

out_face_dir = BREPNET_NPZ_DIR / "embeddings"


brep_metrics = evaluate_brepnet_faces_object_max(
        embeddings_dir=out_face_dir,
        normalize=True
    )

In [ ]:
import pandas as pd
df_metrics = pd.DataFrame(rows)
df_brep = pd.DataFrame([{"model": "BRepNet", **brep_metrics}])
# объединяем результаты

df_metrics = pd.concat([df_metrics, df_brep], ignore_index=True)

cols = ["model", "queries", "recall@1", "recall@5", "recall@10", "mAP", "nDCG@5", "nDCG@10",
        "pos_mean", "neg_mean", "margin", "cohens_d"]

df_metrics = df_metrics.reindex(columns=cols)
df_metrics = df_metrics.sort_values([ "margin", "recall@1", "mAP"], ascending=False)
print(df_metrics.to_string(index=False))


# У brenet выключена нормализация (на pos_mean и neg_mean можно не смотреть, так как эвкливово расстояние не нормализовано)
#         model  queries  recall@1  recall@5  recall@10      mAP   nDCG@5  nDCG@10      pos_mean  neg_mean   margin  cohens_d
#       BRepNet     9895  0.868014  0.999798        1.0 0.927224 0.945822 0.945894 -1.000007e-08 -1.614657 1.614657  1.497281
# DINO s14 2dv0     1032  0.897287  0.996124        1.0 0.943849 0.956899 0.958236  1.000000e+00  0.639031 0.360969  2.803236
#   DINO b14 2d     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904  1.000000e+00  0.827749 0.172251  2.222084
# DINO b14 2dv0     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904  1.000000e+00  0.827749 0.172251  2.222084
#   DINO s14 2d     1032  0.891473  0.998062        1.0 0.939244 0.954075 0.954766  1.000000e+00  0.871053 0.128947  2.246197
# CLIP b32 2dv0     1032  0.890504  0.996124        1.0 0.939731 0.953821 0.955158  1.000000e+00  0.894322 0.105678  2.684414
# CLIP s16 2dv0     1032  0.903101  0.996124        1.0 0.946675 0.958978 0.960314  1.000000e+00  0.906414 0.093586  2.748654
#   CLIP s16 2d     1032  0.975775  1.000000        1.0 0.987403 0.990679 0.990679  1.000000e+00  0.911103 0.088897  2.551635
#   CLIP b32 2d     1032  0.975775  1.000000        1.0 0.987403 0.990679 0.990679  1.000000e+00  0.915726 0.084274  2.541060


# У brenet включена нормализация
#         model  queries  recall@1  recall@5  recall@10      mAP   nDCG@5  nDCG@10  pos_mean  neg_mean   margin  cohens_d
# DINO s14 2dv0     1032  0.897287  0.996124        1.0 0.943849 0.956899 0.958236       1.0  0.639031 0.360969  2.803236
#   DINO b14 2d     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
# DINO b14 2dv0     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
#   DINO s14 2d     1032  0.891473  0.998062        1.0 0.939244 0.954075 0.954766       1.0  0.871053 0.128947  2.246197
#       BRepNet     9895  0.868014  0.999798        1.0 0.927224 0.945822 0.945894       1.0  0.876704 0.123296  0.819112
# CLIP b32 2dv0     1032  0.890504  0.996124        1.0 0.939731 0.953821 0.955158       1.0  0.894322 0.105678  2.684414
# CLIP s16 2dv0     1032  0.903101  0.996124        1.0 0.946675 0.958978 0.960314       1.0  0.906414 0.093586  2.748654
#   CLIP s16 2d     1032  0.975775  1.000000        1.0 0.987403 0.990679 0.990679       1.0  0.911103 0.088897  2.551635
#   CLIP b32 2d     1032  0.975775  1.000000        1.0 0.987403 0.990679 0.990679       1.0  0.915726 0.084274  2.541060

        model  queries  recall@1  recall@5  recall@10      mAP   nDCG@5  nDCG@10  pos_mean  neg_mean   margin  cohens_d
DINO s14 2dv0     1032  0.897287  0.996124        1.0 0.943849 0.956899 0.958236       1.0  0.639031 0.360969  2.803236
  DINO b14 2d     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
DINO b14 2dv0     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
  DINO s14 2d     1032  0.891473  0.998062        1.0 0.939244 0.954075 0.954766       1.0  0.871053 0.128947  2.246197
      BRepNet     9895  0.868014  0.999798        1.0 0.927224 0.945822 0.945894       1.0  0.876704 0.123296  0.819112
CLIP b32 2dv0     1032  0.890504  0.996124        1.0 0.939731 0.953821 0.955158       1.0  0.894322 0.105678  2.684414
CLIP s16 2dv0     1032  0.903101  0.996124        1.0 0.946675 0.958978 0.960314       1.0  0.906414 0.093586  2.748654
  CLIP s16 2d     1032  0.975775  1.0000

In [19]:
from src.modeling.vit.searcher import vit_topk
query_index = 17
top = 20
# print(vit_topk("DINO s14 2d",  dino_s14_2d_files,  query=query_index, k=top).to_string())
# vit_topk("DINO b14 2d",  dino_b14_2d_files,  query=query_index, k=top)
# vit_topk("CLIP s16 2d",  clip_s16_2d_files,  query=query_index, k=top)
# vit_topk("CLIP b32 2d",  clip_b32_2d_files,  query=query_index, k=top)

print(vit_topk("DINO s14 2dv0", dino_s14_2dv0_files, query=query_index, k=top).to_string())
# vit_topk("DINO b14 2dv0", dino_b14_2dv0_files, query=query_index, k=top)
# vit_topk("CLIP s16 2dv0", clip_s16_2dv0_files, query=query_index, k=top)
# vit_topk("CLIP b32 2dv0", clip_b32_2dv0_files, query=query_index, k=top)

Поисковая модель: DINO s14 2dv0
                       model     score
0    42. Silencer Fix-06.prt  1.000000
1    42. Silencer Fix-07.prt  1.000000
2    42. Silencer Fix-05.prt  0.965369
3    42. Silencer Fix-08.prt  0.951780
4    42. Silencer Fix-04.prt  0.951452
5       42. Silencer Fix.prt  0.939669
6    42. Silencer Fix-03.prt  0.936808
7    42. Silencer Fix-01.prt  0.928322
8    42. Silencer Fix-09.prt  0.927563
9    42. Silencer Fix-10.prt  0.924655
10   42. Silencer Fix-02.prt  0.923597
11    Колодка приклада 6.prt  0.707258
12    Колодка приклада 7.prt  0.706994
13    Колодка приклада 9.prt  0.699907
14    Колодка приклада 8.prt  0.699887
15    Колодка приклада 5.prt  0.697851
16    Колодка приклада 4.prt  0.692574
17    Колодка приклада 3.prt  0.692472
18  44. Extractor Pin-01.prt  0.675459
19    Колодка приклада 2.prt  0.664904


In [35]:
from src.modeling.BRepNet.searcher import search_topk
step_files = get_files(PROCESSED_DATA_DIR / "dataset_129" / 'stp')
print(f'Количество файлов STEP: {len(step_files)}')

stp_model = step_files[query_index]
print(f'Запрос: {stp_model.stem}')

df, matches, dmin, interval = search_topk(out_face_dir, query_stem=stp_model.stem, k=top, exclude_self=False)

print(df.to_string())


Количество файлов STEP: 129
Запрос: 42. Silencer Fix-07.prt
                        model      score
0     42. Silencer Fix-07.prt   0.000000
1     42. Silencer Fix-06.prt   0.000000
2     42. Silencer Fix-05.prt   4.318653
3     42. Silencer Fix-04.prt  10.556989
4     42. Silencer Fix-08.prt  10.600834
5     42. Silencer Fix-10.prt  16.423070
6     42. Silencer Fix-09.prt  19.145785
7     42. Silencer Fix-03.prt  20.211120
8    Колодка прицельная 9.prt  20.903475
9        42. Silencer Fix.prt  21.306449
10    42. Silencer Fix-01.prt  22.023930
11    42. Silencer Fix-02.prt  22.048089
12   Колодка прицельная 1.prt  22.983625
13     Колодка прицельная.prt  23.132900
14  Колодка прицельная 10.prt  23.417440
15                  Кожух.prt  24.220148
16                Кожух 6.prt  24.379845
17                Кожух 9.prt  24.405034
18   Колодка прицельная 7.prt  24.643578
19                Кожух 7.prt  24.748889


In [47]:
from src.visualization.jupyter_segmentation_viewer import JupyterSegmentationViewer

def find_step_path_by_stem(step_files: list[Path], stem_name: str):
    for f in step_files:
        if f.stem == stem_name:
            return f

query_index_top = 8

model_name = df.iloc[query_index_top]["model"]
step_path = find_step_path_by_stem(step_files, model_name)
print(step_path)
viewer_query = JupyterSegmentationViewer(stp_model)
viewer_query.view_solid()

if step_path is not None: 
    viewer_pairs = JupyterSegmentationViewer(step_path)
    
    # Извлекаем совпадения для лучшего результата по индексу
    best_match_data = matches[query_index_top] 
    
    # Создаем массив расстояний для целевой модели
    num_target_faces = len(list(viewer_pairs.solid.faces())) 
    target_dists = np.full(num_target_faces, np.inf) # Заполняем inf, чтобы ненайденные грани были видны
    
    target_indices = best_match_data[:, 1].astype(int)
    distances = best_match_data[:, 2]
    
    # Для каждой целевой грани сохраняем минимальное расстояние от любой из граней запроса
    np.minimum.at(target_dists, target_indices, distances)
    
    # Заменяем inf на максимальное значение в интервале для корректного отображения
    target_dists[target_dists == np.inf] = interval[1]

    viewer_pairs.display_faces_with_heatmap(target_dists, interval)


D:\workspace\projects\freelance\Fusion3DNet\data\processed\dataset_129\stp\Колодка прицельная 9.prt.stp


TraitError: The 'rotation' trait of a GridHelper instance contains an Enum of an Euler which expected any of ['XYZ', 'YZX', 'ZXY', 'XZY', 'YXZ', 'ZYX'], not the str 'xyz'.

TraitError: The 'rotation' trait of a GridHelper instance contains an Enum of an Euler which expected any of ['XYZ', 'YZX', 'ZXY', 'XZY', 'YXZ', 'ZYX'], not the str 'xyz'.